Instalando Bibliotecas

In [14]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 3.8 MB/s eta 0:00:00


In [15]:
import pandas as pd
import unidecode

In [2]:
# Substitua 'seu_arquivo.json' pelo caminho para o seu arquivo JSON
caminho_do_arquivo = '/content/applicants.json'

try:
    # Lê o arquivo JSON e o converte para um DataFrame
    # A função read_json do pandas é muito versátil e pode lidar com diferentes estruturas JSON
    df = pd.read_json(caminho_do_arquivo)

    # Exibe o DataFrame em formato de tabela
    print(df)

except FileNotFoundError:
    print(f"Erro: O arquivo '{caminho_do_arquivo}' não foi encontrado.")
except Exception as e:
    print(f"Ocorreu um erro ao ler o arquivo: {e}")

                                                                       31000  \
infos_basicas              {'telefone_recado': '', 'telefone': '(11) 9704...   
informacoes_pessoais       {'data_aceite': 'Cadastro anterior ao registro...   
informacoes_profissionais  {'titulo_profissional': '', 'area_atuacao': ''...   
formacao_e_idiomas         {'nivel_academico': '', 'nivel_ingles': '', 'n...   
cargo_atual                                                               {}   
cv_pt                      assistente administrativo\n\n\nsantosbatista\n...   
cv_en                                                                          

                                                                       31001  \
infos_basicas              {'telefone_recado': '', 'telefone': '(11) 9372...   
informacoes_pessoais       {'data_aceite': 'Cadastro anterior ao registro...   
informacoes_profissionais  {'titulo_profissional': 'Analista Administrati...   
formacao_e_idiomas         {'nivel_acad

In [19]:
# Substitua 'seu_arquivo.json' pelo caminho para o seu arquivo JSON
caminho_do_arquivo = '/content/applicants.json'

def detectar_genero(nome):
    """
    Detecta gênero a partir do nome.
    Regra simples: nomes terminados em 'a' (exceto 'ma') = feminino, caso contrário masculino.
    """
    if not isinstance(nome, str) or nome.strip() == "":
        return "Masculino"  # default
    nome = unidecode.unidecode(nome.strip().lower())
    if nome.endswith("a") and not nome.endswith("ma"):
        return "Feminino"
    return "Masculino"

def ajustar_estado_civil(estado_civil, genero):
    """
    Ajusta o estado civil conforme o gênero detectado.
    """
    if not isinstance(estado_civil, str):
        return estado_civil

    estado_civil = estado_civil.strip().capitalize()
    mapa = {
        "Solteiro": {"Masculino": "Solteiro", "Feminino": "Solteira"},
        "Casado": {"Masculino": "Casado", "Feminino": "Casada"},
        "Divorciado": {"Masculino": "Divorciado", "Feminino": "Divorciada"},
        "Viúvo": {"Masculino": "Viúvo", "Feminino": "Viúva"},
    }

    for base, genero_dict in mapa.items():
        if estado_civil.startswith(base[:-1]):  # trata "Solteir"/"Solteiro"/"Solteira"
            return genero_dict[genero]
    return estado_civil  # mantém se não encontrar no mapa

try:
    # Lê o arquivo JSON e o converte para um DataFrame
    df_applicants = pd.read_json(caminho_do_arquivo)

    # Lista para armazenar os dados extraídos
    dados_extraidos = []

    # Itera sobre as colunas (cada candidato é uma coluna)
    for col in df_applicants.columns:
        candidato_info = {'id_candidato': col}

        for row_index in df_applicants.index:
            valor = df_applicants.loc[row_index, col]

            if isinstance(valor, dict):
                for key, value in valor.items():
                    candidato_info[f'{row_index}_{key}'] = value
            else:
                candidato_info[row_index] = valor

        dados_extraidos.append(candidato_info)

    # Cria DataFrame final
    df_final = pd.DataFrame(dados_extraidos)

    # === Correção de gênero e estado civil ===
    if all(col in df_final.columns for col in ["infos_basicas_nome",
                                               "informacoes_pessoais_sexo",
                                               "informacoes_pessoais_estado_civil"]):
        for idx, row in df_final.iterrows():
            genero_detectado = detectar_genero(row["infos_basicas_nome"])
            df_final.at[idx, "informacoes_pessoais_sexo"] = genero_detectado
            df_final.at[idx, "informacoes_pessoais_estado_civil"] = ajustar_estado_civil(
                row["informacoes_pessoais_estado_civil"], genero_detectado
            )

    display(df_final)

except FileNotFoundError:
    print(f"Erro: O arquivo '{caminho_do_arquivo}' não foi encontrado.")
except Exception as e:
    print(f"Ocorreu um erro ao ler o arquivo: {e}")


,id_candidato,infos_basicas_telefone_recado,infos_basicas_telefone,infos_basicas_objetivo_profissional,infos_basicas_data_criacao,infos_basicas_inserido_por,infos_basicas_email,infos_basicas_local,infos_basicas_sabendo_de_nos_por,infos_basicas_data_atualizacao,...,cargo_atual_id_ibrati,cargo_atual_email_corporativo,cargo_atual_cargo_atual,cargo_atual_projeto_atual,cargo_atual_cliente,cargo_atual_unidade,cargo_atual_data_admissao,cargo_atual_data_ultima_promocao,cargo_atual_nome_superior_imediato,cargo_atual_email_superior_imediato
0,31000,,(11) 97048-2708,,10-11-2021 07:29:49,Luna Correia,carolina_aparecida@gmail.com,,,10-11-2021 07:29:49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,31001,,(11) 93723-4396,Analista Administrativo,10-11-2021 08:56:16,Laura Pacheco,eduardo_rios@hotmail.com,"São Paulo, São Paulo",Outros,11-11-2021 11:10:31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,31002,,(11) 92399-9824,Administrativo | Financeiro,10-11-2021 09:01:00,Laura Pacheco,pedro_henrique_carvalho@gmail.com,"São Paulo, São Paulo",Anúncio,10-11-2021 11:42:36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,31003,,(11) 98100-1727,Área administrativa,10-11-2021 09:08:13,Laura Pacheco,thiago_barbosa@hotmail.com,"São Paulo, São Paulo",Site de Empregos,10-11-2021 16:04:51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,31004,,(11) 92517-2678,,10-11-2021 09:18:46,Maria Clara Pires,diogo_das_neves@hotmail.com,,,10-11-2021 09:22:03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42477,5995,,(11) 92622-2088,Crm / Líder de Projetos,14-03-2018 16:35:14,pelo próprio candidato,gabrielly_novaes@gmail.com,,Outros,14-03-2018 16:35:14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42478,5996,,(41) 93009-5141,Crm,14-03-2018 16:38:28,pelo próprio candidato,sr._gabriel_rios@gmail.com,,Outros,14-03-2018 16:38:28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42479,5997,,(11) 97837-3941,Crm,14-03-2018 16:41:52,pelo próprio candidato,dr._mathias_da_cunha@gmail.com,,Outros,14-03-2018 16:41:52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42480,5998,,(61) 96806-8075,Crm,14-03-2018 16:42:49,pelo próprio candidato,bruna_vieira@hotmail.com,,Outros,14-03-2018 16:42:49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# Substitua 'seu_arquivo.json' pelo caminho para o seu arquivo JSON
caminho_do_arquivo = '/content/prospects.json'

try:
    # Lê o arquivo JSON e o converte para um DataFrame
    # A função read_json do pandas é muito versátil e pode lidar com diferentes estruturas JSON
    df = pd.read_json(caminho_do_arquivo)

    # Exibe o DataFrame em formato de tabela
    print(df)

except FileNotFoundError:
    print(f"Erro: O arquivo '{caminho_do_arquivo}' não foi encontrado.")
except Exception as e:
    print(f"Ocorreu um erro ao ler o arquivo: {e}")

                                                        4530   \
titulo                                    CONSULTOR CONTROL M   
modalidade                                                      
prospects   [{'nome': 'José Vieira', 'codigo': '25632', 's...   

                                                        4531  4532   \
titulo      2021-2607395-PeopleSoft Application Engine-Dom...         
modalidade                                                            
prospects   [{'nome': 'Sra. Yasmin Fernandes', 'codigo': '...    []   

                                                        4533   \
titulo      2021-2605708-Microfocus Application Life Cycle...   
modalidade                                                      
prospects   [{'nome': 'Arthur Almeida', 'codigo': '26338',...   

                                                        4534   \
titulo      2021-2605711-Microfocus QTP - UFT Automation T...   
modalidade                                                     

In [21]:
# Substitua 'seu_arquivo.json' pelo caminho para o seu arquivo JSON
caminho_do_arquivo = '/content/prospects.json'

try:
    # Lê o arquivo JSON e o converte para um DataFrame
    df_prospects = pd.read_json(caminho_do_arquivo)

    # Lista para armazenar os dados extraídos
    dados_prospects = []

    # Itera sobre as colunas (que representam as oportunidades)
    for col in df_prospects.columns:
        titulo = df_prospects.loc['titulo', col]
        modalidade = df_prospects.loc['modalidade', col]
        prospects_list = df_prospects.loc['prospects', col]

        # Itera sobre a lista de prospects para cada oportunidade
        if isinstance(prospects_list, list):
            for prospect in prospects_list:
                if isinstance(prospect, dict):
                    dados_prospects.append({
                        'codigo': prospect.get('codigo'),
                        'titulo': titulo,
                        'modalidade': modalidade,
                        'nome': prospect.get('nome')
                    })

    # Cria um novo DataFrame a partir dos dados extraídos
    df_prospects_final = pd.DataFrame(dados_prospects)

    # Exibe o novo DataFrame
    display(df_prospects_final)

except FileNotFoundError:
    print(f"Erro: O arquivo '{caminho_do_arquivo}' não foi encontrado.")
except Exception as e:
    print(f"Ocorreu um erro ao ler o arquivo: {e}")

,codigo,titulo,modalidade,nome
0,25632,CONSULTOR CONTROL M,,José Vieira
1,25529,CONSULTOR CONTROL M,,Srta. Isabela Cavalcante
2,25364,2021-2607395-PeopleSoft Application Engine-Dom...,,Sra. Yasmin Fernandes
3,25360,2021-2607395-PeopleSoft Application Engine-Dom...,,Alexia Barbosa
4,26338,2021-2605708-Microfocus Application Life Cycle...,,Arthur Almeida
...,...,...,...,...
53754,2018,SAP SD,,Marcelo Azevedo
53755,40384,Java Support Analyst,,Dr. Vitor Hugo Silva
53756,16828,Consultor Sênior Especialista SAP LES-TRA - 1433,,Ana Cardoso
53757,15042,Consultor Sênior Especialista SAP LES-TRA - 1433,,Pedro Lucas das Neves


In [28]:
# Substitua 'seu_arquivo.json' pelo caminho para o seu arquivo JSON
caminho_do_arquivo = '/content/vagas.json'

try:
    # Tenta ler o arquivo JSON diretamente
    df = pd.read_json(caminho_do_arquivo)

    # Exibe o DataFrame em formato de tabela
    print(df)

except Exception as e:
    print(f"Ocorreu um erro ao ler o arquivo: {e}")
    # Se ocorrer um erro, tenta ler o arquivo como texto e lidar com possíveis problemas
    try:
        with open(caminho_do_arquivo, 'r', encoding='utf-8') as f:
            content = f.read()

        # Tenta corrigir o problema da aspa dupla extra (exemplo: substituir por aspa simples ou remover)
        # Esta é uma solução genérica, pode precisar ser ajustada dependendo do erro exato no JSON
        content_corrected = content.replace('""', '"') # Exemplo: substitui aspas duplas consecutivas por uma única aspa

        # Tenta ler o conteúdo corrigido como JSON
        from io import StringIO
        df = pd.read_json(StringIO(content_corrected))

        # Exibe o DataFrame em formato de tabela
        print("DataFrame lido após correção:")
        print(df)

    except FileNotFoundError:
        print(f"Erro: O arquivo '{caminho_do_arquivo}' não foi encontrado.")
    except Exception as e_corrected:
        print(f"Ocorreu um erro ao ler o arquivo mesmo após correção: {e_corrected}")

                                                                 5185   \
informacoes_basicas  {'data_requicisao': '04-05-2021', 'limite_espe...   
perfil_vaga          {'pais': 'Brasil', 'estado': 'São Paulo', 'cid...   
beneficios           {'valor_venda': '-', 'valor_compra_1': 'R$', '...   

                                                                 5184   \
informacoes_basicas  {'data_requicisao': '04-05-2021', 'limite_espe...   
perfil_vaga          {'pais': 'Brasil', 'estado': 'São Paulo', 'cid...   
beneficios           {'valor_venda': '-', 'valor_compra_1': 'R$', '...   

                                                                 5183   \
informacoes_basicas  {'data_requicisao': '04-05-2021', 'limite_espe...   
perfil_vaga          {'pais': 'Brasil', 'estado': 'São Paulo', 'cid...   
beneficios           {'valor_venda': '-', 'valor_compra_1': 'R$', '...   

                                                                 5182   \
informacoes_basicas  {'data_requici

In [29]:
# Substitua 'seu_arquivo.json' pelo caminho para o seu arquivo JSON
caminho_do_arquivo = '/content/vagas.json'

try:
    # Tenta ler o arquivo JSON diretamente
    df_vagas = pd.read_json(caminho_do_arquivo)

    # Transpõe o DataFrame para ter as informações como colunas e os IDs das vagas como linhas
    df_vagas_transposed = df_vagas.transpose()

    # Exibe o novo DataFrame
    display(df_vagas_transposed)

except Exception as e:
    print(f"Ocorreu um erro ao ler o arquivo: {e}")
    # Se ocorrer um erro, tenta ler o arquivo como texto e lidar com possíveis problemas
    try:
        with open(caminho_do_arquivo, 'r', encoding='utf-8') as f:
            content = f.read()

        # Tenta corrigir o problema da aspa dupla extra (exemplo: substituir por aspa simples ou remover)
        # Esta é uma solução genérica, pode precisar ser ajustada dependendo do erro exato no JSON
        content_corrected = content.replace('""', '"') # Exemplo: substitui aspas duplas consecutivas por uma única aspa

        # Tenta ler o conteúdo corrigido como JSON
        from io import StringIO
        df_vagas = pd.read_json(StringIO(content_corrected))

        # Transpõe o DataFrame para ter as informações como colunas e os IDs das vagas como linhas
        df_vagas_transposed = df_vagas.transpose()

        # Exibe o novo DataFrame
        print("DataFrame lido após correção:")
        display(df_vagas_transposed)

    except FileNotFoundError:
        print(f"Erro: O arquivo '{caminho_do_arquivo}' não foi encontrado.")
    except Exception as e_corrected:
        print(f"Ocorreu um erro ao ler o arquivo mesmo após correção: {e_corrected}")

,informacoes_basicas,perfil_vaga,beneficios
5185,"{'data_requicisao': '04-05-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '-', 'valor_compra_1': 'R$', '..."
5184,"{'data_requicisao': '04-05-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '-', 'valor_compra_1': 'R$', '..."
5183,"{'data_requicisao': '04-05-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '-', 'valor_compra_1': 'R$', '..."
5182,"{'data_requicisao': '04-05-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '- p/ mês (168h)', 'valor_comp..."
5181,"{'data_requicisao': '04-05-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '-', 'valor_compra_1': 'R$', '..."
...,...,...,...
12368,"{'data_requicisao': '28-02-2024', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '123,00 -', 'valor_compra_1': ..."
12367,"{'data_requicisao': '28-02-2024', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '168 -', 'valor_compra_1': 'Fe..."
12366,"{'data_requicisao': '27-02-2024', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '168 -', 'valor_compra_1': 'Fe..."
12365,"{'data_requicisao': '27-02-2024', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '168 -', 'valor_compra_1': 'Fe..."


In [30]:
# Substitua 'seu_arquivo.json' pelo caminho para o seu arquivo JSON
caminho_do_arquivo = '/content/vagas.json'

try:
    # Tenta ler o arquivo JSON diretamente
    df_vagas = pd.read_json(caminho_do_arquivo)

    # Transpõe o DataFrame para ter as informações como colunas e os IDs das vagas como linhas
    df_vagas_transposed = df_vagas.transpose()

    # Lista para armazenar os dados extraídos
    dados_vagas_extraidos = []

    # Itera sobre as linhas do DataFrame transposto (que agora representam as vagas)
    for index, row in df_vagas_transposed.iterrows():
        vaga_info = {}
        vaga_info['ID da Vaga'] = index  # Adiciona o ID da vaga

        # Itera sobre as colunas (que agora representam as categorias de informação)
        for col in df_vagas_transposed.columns:
            # Verifica se o valor é um dicionário
            if isinstance(row[col], dict):
                # Itera sobre os itens do dicionário e adiciona ao dicionário da vaga
                for key, value in row[col].items():
                    vaga_info[f'{col}_{key}'] = value
            else:
                # Se não for um dicionário, adiciona o valor diretamente
                vaga_info[col] = row[col]

        dados_vagas_extraidos.append(vaga_info)

    # Cria um novo DataFrame a partir dos dados extraídos
    df_vagas_final = pd.DataFrame(dados_vagas_extraidos)

    # Exibe o novo DataFrame
    display(df_vagas_final)

except Exception as e:
    print(f"Ocorreu um erro ao ler o arquivo: {e}")
    # Se ocorrer um erro, tenta ler o arquivo como texto e lidar com possíveis problemas
    try:
        with open(caminho_do_arquivo, 'r', encoding='utf-8') as f:
            content = f.read()

        # Tenta corrigir o problema da aspa dupla extra (exemplo: substituir por aspa simples ou remover)
        # Esta é uma solução genérica, pode precisar ser ajustada dependendo do erro exato no JSON
        content_corrected = content.replace('""', '"') # Exemplo: substitui aspas duplas consecutivas por uma única aspa

        # Tenta ler o conteúdo corrigido como JSON
        from io import StringIO
        df_vagas = pd.read_json(StringIO(content_corrected))

        # Transpõe o DataFrame para ter as informações como colunas e os IDs das vagas como linhas
        df_vagas_transposed = df_vagas.transpose()

        # Lista para armazenar os dados extraídos
        dados_vagas_extraidos = []

        # Itera sobre as linhas do DataFrame transposto (que agora representam as vagas)
        for index, row in df_vagas_transposed.iterrows():
            vaga_info = {}
            vaga_info['ID da Vaga'] = index  # Adiciona o ID da vaga

            # Itera sobre as colunas (que agora representam as categorias de informação)
            for col in df_vagas_transposed.columns:
                # Verifica se o valor é um dicionário
                if isinstance(row[col], dict):
                    # Itera sobre os itens do dicionário e adiciona ao dicionário da vaga
                    for key, value in row[col].items():
                        vaga_info[f'{col}_{key}'] = value
                else:
                    # Se não for um dicionário, adiciona o valor diretamente
                    vaga_info[col] = row[col]

            dados_vagas_extraidos.append(vaga_info)

        # Cria um novo DataFrame a partir dos dados extraídos
        df_vagas_final = pd.DataFrame(dados_vagas_extraidos)

        # Exibe o novo DataFrame
        print("DataFrame lido após correção:")
        display(df_vagas_final)

    except FileNotFoundError:
        print(f"Erro: O arquivo '{caminho_do_arquivo}' não foi encontrado.")
    except Exception as e_corrected:
        print(f"Ocorreu um erro ao ler o arquivo mesmo após correção: {e_corrected}")

,ID da Vaga,informacoes_basicas_data_requicisao,informacoes_basicas_limite_esperado_para_contratacao,informacoes_basicas_titulo_vaga,informacoes_basicas_vaga_sap,informacoes_basicas_cliente,informacoes_basicas_solicitante_cliente,informacoes_basicas_empresa_divisao,informacoes_basicas_requisitante,informacoes_basicas_analista_responsavel,...,perfil_vaga_demais_observacoes,perfil_vaga_viagens_requeridas,perfil_vaga_equipamentos_necessarios,beneficios_valor_venda,beneficios_valor_compra_1,beneficios_valor_compra_2,informacoes_basicas_data_inicial,informacoes_basicas_data_final,perfil_vaga_habilidades_comportamentais_necessarias,informacoes_basicas_nome_substituto
0,5185,04-05-2021,00-00-0000,Operation Lead -,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,...,100% Remoto Período – entre 5 – 6 meses,,Nenhum -,-,R$,,NaN,NaN,NaN,NaN
1,5184,04-05-2021,00-00-0000,Consultor PP/QM Sênior,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Yasmin da Rosa,...,• Início: Imediato • Fim: Jan/22,,Nenhum -,-,R$,,NaN,NaN,NaN,NaN
2,5183,04-05-2021,00-00-0000,ANALISTA PL/JR C/ SQL,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Ana Albuquerque,...,Localização: Remoto Perfil: Analista Pleno ou ...,,Nenhum -,-,R$,,NaN,NaN,NaN,NaN
3,5182,04-05-2021,18-05-2021,Technical Architect - 11894809,Não,Nelson-Page,Dr. Raul Monteiro,Decision São Paulo,Cecília Freitas,Clara Rios,...,Budgeted Rate - indicate currency and type (ho...,Não,Notebook padrão -,- p/ mês (168h),fechado,,18-05-2021,17-01-2022,NaN,NaN
4,5181,04-05-2021,00-00-0000,Consultor SAP AUTHORIZATION (BCA) -Pleno / Sênior,Não,Mann and Sons,Cauê Fogaça,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,...,contratação CLT full pela Decision locação rem...,Sim,Nenhum -,-,R$,,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14076,12368,28-02-2024,31-03-2024,PMP 5203658 - Dynamics,Não,Miller-Curry,Guilherme Campos,Decision Campinas,Sra. Juliana Oliveira,Sra. Joana Sousa,...,,,,"123,00 -",APPLICATION DEVELOPER-MICROSOFT ANALYTICS - ba...,,NaN,NaN,NaN,NaN
14077,12367,28-02-2024,29-02-2024,MM com IM - FSR 3910,Não,Bishop-Reed,Lívia Vasconcelos,Decision São Paulo,Manuella Jesus,Srta. Bella Ferreira,...,FSR: 3910,Não,,168 -,Fechado,,NaN,NaN,NaN,NaN
14078,12366,27-02-2024,29-02-2024,964 - Assistente fiscal ou financeiro,Não,"Glover, Warren and Norris",Lorenzo Cunha,Decision São Paulo,Maria Lopes,Maria Clara Pires,...,"Modelo de trabalho: Hibrido, 3 vezes por seman...",Não,,168 -,Fechado,,NaN,NaN,NaN,NaN
14079,12365,27-02-2024,29-02-2024,966 - Analista pleno ou sênior de área fiscal,Não,"Glover, Warren and Norris",Lorenzo Cunha,Decision São Paulo,Maria Lopes,Melina Montenegro,...,"Modelo de trabalho: Hibrido, 3 vezes por seman...",Não,,168 -,Fechado,,NaN,NaN,NaN,NaN


# Gerando dataset com os arquivos tratados

In [31]:
# Exportar df_final para CSV
df_final.to_csv("/content/applicants.csv", index=False, encoding="utf-8-sig")

# Exportar df_prospects_final para CSV
df_prospects_final.to_csv("/content/prospects.csv", index=False, encoding="utf-8-sig")

# Exportar df_vagas_final para CSV
df_vagas_final.to_csv("/content/vagas.csv", index=False, encoding="utf-8-sig")

print("DataFrames exportados para CSV com sucesso!")

DataFrames exportados para CSV com sucesso!
